In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import sys

In [ ]:
def convert_format(arr, N):
    ret = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            ret[i,j] = arr[i + N * j]
    
    return ret

def read_snapshots(N):
    snapshots = []
    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format(np.fromfile(f"phi1-snapshot{count}", dtype=np.float32), N)
            phi2 = convert_format(np.fromfile(f"phi2-snapshot{count}", dtype=np.float32), N)
            # saxion = np.sqrt(phi1**2 + phi2**2)
            axion = np.arctan2(phi1, phi2)
            snapshots.append(axion)
            count += 1
        except:
            done = True
    
    return snapshots


In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [13, 12]

snapshots = read_snapshots(256)

for i in range(len(snapshots)):
    plt.imshow(snapshots[i], cmap=cm.twilight)
    plt.title(f"saxion snapshot {i}")
    plt.colorbar()
    plt.show()
